In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [3]:
from test_model import test
from argparse import Namespace
import glob
import numpy as np
from concurrent import futures
import multiprocessing
import copy
import json

In [3]:
args = {
    'data_path': '../Data/',
    'pretrained_model': 'resnet',
    'pretrained_path': '',
    'window_size': 29,
    'strided': 15,
    'batch_size': 512,
    'num_workers': 8,
    'trial_id': 1,
    'ckpt' : 50, #200,
    'car_model': None
}

args = Namespace(**args)

In [4]:
bs = 512
# pretrained_paths = sorted(glob.glob(f'./save/SupCon_resnet18*_bs{bs}*'))
pretrained_paths = sorted(glob.glob('./save/resnet18?_*'))

pretrained_paths

['./save/resnet181_lr0.0005_bs256_50epochs_050122_093024_cosine',
 './save/resnet182_lr0.0005_bs256_50epochs_043022_095208_cosine',
 './save/resnet183_lr0.0005_bs256_50epochs_043022_095431_cosine',
 './save/resnet184_lr0.0005_bs256_50epochs_043022_211030_cosine',
 './save/resnet185_lr0.0005_bs256_50epochs_043022_211029_cosine',
 './save/resnet185_lr0.0005_bs256_50epochs_050122_092956_cosine']

In [5]:
num_cpu = multiprocessing.cpu_count()
workers = max(num_cpu, len(pretrained_paths))

with futures.ProcessPoolExecutor(workers) as executor:
    to_do = []
    for path in pretrained_paths:
        args.trial_id = path.split('_')[0][-1]
        args.pretrained_path = f'{path}/models/'
        future = executor.submit(test, copy.deepcopy(args), 
                                 {'verbose': False, 'is_cuda': True})
        to_do.append(future)
        
    
    total_results = {}
    for future in futures.as_completed(to_do):
        try:
            results = future.result()
            for k, v in results.items():
                total_results.setdefault(k, [])
                total_results[k].append(v)
        except Exception as error:
            print('An exception occurred: {}'.format(error))

Loading:  SupCEResNet
Loading:  Loading: SupCEResNet Loading: 
SupCEResNet 
SupCEResNetLoading: 
 SupCEResNet
Loading:  SupCEResNet


  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

['0.0039', '0.1749', '0.1240', '0.1373', '0.0966']
['1.0000', '0.9983', '0.9988', '0.9986', '0.9990']
['0.9993', '1.0000', '0.9998', '0.9998', '0.9999']
['0.9996', '0.9991', '0.9993', '0.9992', '0.9995']
['0.0020', '0.1934', '0.1705', '0.1321', '0.1063']
['1.0000', '0.9981', '0.9983', '0.9987', '0.9989']
['0.9992', '1.0000', '0.9999', '0.9999', '0.9999']
['0.9996', '0.9990', '0.9991', '0.9993', '0.9994']
['0.0024', '0.2118', '0.1705', '0.1004', '0.1015']
['1.0000', '0.9979', '0.9983', '0.9990', '0.9990']
['0.9992', '1.0000', '0.9998', '0.9998', '0.9999']
['0.9996', '0.9989', '0.9991', '0.9994', '0.9994']
['0.0024', '0.1151', '0.2248', '0.0924', '0.0821']
['1.0000', '0.9988', '0.9978', '0.9991', '0.9992']
['0.9993', '1.0000', '0.9999', '0.9998', '0.9998']
['0.9997', '0.9994', '0.9988', '0.9994', '0.9995']
['0.0005', '0.2302', '0.2015', '0.0819', '0.0918']
['1.0000', '0.9977', '0.9980', '0.9992', '0.9991']
['0.9992', '0.9999', '0.9999', '0.9998', '0.9999']
['0.9996', '0.9988', '0.9989', 

In [6]:
print('Final results')
total_results = {k: np.stack(v, axis=0) for k, v in total_results.items()}
for k, v in total_results.items():
    print(k, list("{0:0.4f}".format(i) for i in v.mean(axis=0)))

Final results
fnr ['0.0020', '0.1880', '0.1867', '0.1012', '0.1003']
rec ['1.0000', '0.9981', '0.9981', '0.9990', '0.9990']
pre ['0.9992', '0.9999', '0.9999', '0.9998', '0.9999']
f1 ['0.9996', '0.9990', '0.9990', '0.9994', '0.9994']


In [7]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [8]:
file = f'{args.pretrained_model}{bs}.json'
with open(f'./reports/{file}', 'w') as convert_file:
     convert_file.write(json.dumps(total_results, cls=NumpyEncoder))